In [2]:
import numpy as np
from numpy import float64
from numpy.typing import NDArray, ArrayLike
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sci
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
class Rede_Neural:
    def __init__(self, qtd_neuronios_camada, seed) -> None:
        self.camadas: int = len(qtd_neuronios_camada)
        self.neuronios_camada: list[int] = qtd_neuronios_camada
        self.pesos = [self.pesos_iniciais(seed)]
        self.seed = seed
        self.perdas = []

    def ativacao(self, z: ArrayLike) -> NDArray:
        """
        Aplica a função de ativação definida em uma lista ou np.array. Atualmente é tanh, pq ReLU e Leaku ReLU é linear demais pro que queremos.
        """
        return 5 * np.tanh(z/5)

    def derivada_ativacao(self, z: ArrayLike) -> NDArray:
        """
        Aplica a derivada da função de ativação definida em uma lista ou np.array. Atualmente é leaky ReLU, com 0.01 se z < 0
        """
        return 1 / np.cosh(z/5) ** 2

    def pesos_iniciais(self, seed: int) -> list[NDArray[float64]]:
        """
        Usa a distribuição de Xavier para criar pesos iniciais usando a seed dada para o rng.\n
        W(camada) ~ N(0, sqrt(2 / neuronios[camada]))
        """
        pesos_saida = []
        np.random.seed(seed)

        # Inicializacao de Kaiming
        # for i in range(self.camadas - 1):
        #     n_in = self.neuronios_camada[i]
        #     n_out = self.neuronios_camada[i+1]
        #     pesos = np.random.normal(
        #         0,
        #         np.sqrt(2/n_in),
        #         size=(n_out, n_in+1)
        #     )
        #     pesos_saida.append(pesos)

        # Inicializacao de Xavier
        for i in range(self.camadas - 1):
            n_in = self.neuronios_camada[i]
            n_out = self.neuronios_camada[i+1]
            pesos = np.random.normal(
                0,
                np.sqrt(2 / (n_in + n_out)),
                size=(n_out, n_in+1)
            )
            pesos_saida.append(pesos)

        return pesos_saida

    def forward_pass(self, entrada: ArrayLike, pesos: list[NDArray]) -> list[dict[str, NDArray]]:
        """
        Aplica a rotina de multiplicar pelos pesos e aplicar função de ativação para todas as camadas, retornando os valores pré-ativação e pós-ativação de cada camada.

        Parameters
        ----------
        entrada: ArrayLike
            É a lista de valores de entrada da rede, pode ser uma lista mesmo ou qualquer ArrayLike, já que é transformada em NDArray dentro da função.

        pesos: list[NDArray]
            São os pesos que se quer usar na rede. Usar self.pesos[-1] para última época

        Returns
        -------
        valores_camadas: list[dict]
            Para uma camada qualquer ``i``, os valores de ``dicionário valores_camadas[i]`` são:

            - ``"z"``: NDArray or None\n
                Valores não ativados da camada
            - ``"a"``: NDArray\n
                Valores ativados da camada
        """
        valores_camadas = []
        a = np.array(entrada)
        valores_camadas.append({"z": None, "a": a})

        for W in pesos:
            z = W @ np.append(a, 1) # adiciona bias
            a = self.ativacao(z)
            valores_camadas.append({"z": z, "a": a})
        
        return valores_camadas
    
    def previsao(self, entrada: ArrayLike, epoca: int) -> NDArray:
        """Retorna os valores previstos pela rede, os valores ativados da última camada, para a época desejada. Usar -1 para última época"""
        camadas  = self.forward_pass(entrada, self.pesos[epoca])
        previsao = camadas[-1]["a"]
        return previsao

    def perda(self, previsao: NDArray, real: ArrayLike) -> float:
        """
        Calcula a função de perda da rede, atualmente usando 0.5 MSE mas poderia ser com penalidades e
        para função de valor máximo talvez seja melhor usar uma função como softmax
        """
        residuos: NDArray = previsao - real
        mse = np.mean(residuos ** 2)
        perda = 0.5 * mse
        return perda
    
    def derivada_perda(self, previsao: NDArray, real: ArrayLike) -> NDArray:
        """
        Calcula a derivada da função de perda, lembrar que a derivada de abs(x) = sign(x)
        """
        residuos: NDArray = previsao - real
        return residuos
    
    def calcular_gradiente(self, entrada: ArrayLike, real: ArrayLike, pesos: list[NDArray]) -> list[NDArray]:
        """Calcula o gradiente dos pesos para os pesos dados, retornando uma lista de NDArrays correspondente ao gradiente de cada camada."""
        valores_camadas = self.forward_pass(entrada, pesos)
        previsao = valores_camadas[-1]["a"]
        
        # alocacao de listas
        delta = [None] * (self.camadas - 1)
        gradiente = [None] * (self.camadas - 1)
    
        # delta da ultima camada
        erro_saida = self.derivada_perda(previsao, real)
        delta[-1] = erro_saida * self.derivada_ativacao(valores_camadas[-1]["z"])

        # gradiente da ultima camada
        a_anterior = np.append(valores_camadas[-2]["a"], 1)
        gradiente[-1] = np.outer(delta[-1], a_anterior)

        # gradiente das camadas ocultas
        for camada in reversed(range(self.camadas - 2)):
            # delta da camada
            W_sem_bias = pesos[camada + 1][:, :-1] # descarta coluna do bias
            delta[camada] = (W_sem_bias.T @ delta[camada + 1]) * self.derivada_ativacao(valores_camadas[camada + 1]["z"])
            
            # gradiente da camada
            a_anterior = np.append(valores_camadas[camada]["a"], 1)
            gradiente[camada] = np.outer(delta[camada], a_anterior)

        return gradiente

    def treinar_uma_epoca(self, taxa_aprendizado: float, tamanho_lotes: int, entradas_treino: list[ArrayLike], saidas_treino: list[ArrayLike]) -> None:
        """Calcula os gradientes e corrige os pesos de acordo com os parâmetros dados, salvando os novos pesos em self.pesos. Usando Adam do TensorFlow"""
        # embaralha os indices
        n = len(entradas_treino)
        indices = np.arange(n)
        np.random.shuffle(indices)
        perdas_lote = []

        # pesos para atualizar e inicializa adam
        tf_pesos = [tf.Variable(W, dtype=tf.float32) for W in self.pesos[-1]] # usa a ultima epoca
        adam = tf.keras.optimizers.Adam(learning_rate=taxa_aprendizado)

        # percorre os lotes
        for inicio in range(0, n, tamanho_lotes):
            fim = inicio + tamanho_lotes
            indices_lote = indices[inicio:fim]

            # inicializacao dos pesos e gradientes
            pesos_numpy = [w.numpy() for w in tf_pesos]
            gradiente_acumulado = [np.zeros_like(W) for W in pesos_numpy]

            # calcula o gradiente e soma no gradiente acumulado
            for i in indices_lote:
                entrada = np.array(entradas_treino[i], dtype=np.float32)
                saida_real = np.array(saidas_treino[i], dtype=np.float32)

                gradiente = self.calcular_gradiente(entrada, saida_real, pesos_numpy)
                previsao = self.previsao(entrada, -1)
                perdas_lote.append(self.perda(previsao, saida_real))

                for j in range(self.camadas - 1):
                    gradiente_acumulado[j] += gradiente[j]

            # tira o gradiente medio
            gradiente_medio = [g / len(indices_lote) for g in gradiente_acumulado]
            
            # converte gradiente pra tf.tensor e aplica Adam
            tf_gradientes = [tf.convert_to_tensor(g, dtype=tf.float32) for g in gradiente_medio]
            adam.apply_gradients(zip(tf_gradientes, tf_pesos))
        
        # sumario da epoca
        perda_epoca = np.mean(perdas_lote)
        pesos_numpy = [w.numpy() for w in tf_pesos]
        
        self.pesos.append(pesos_numpy)
        self.perdas.append(perda_epoca)

    def treinar(self, taxa_aprendizado: float, tamanho_lotes: int, entradas_treino: list[ArrayLike], saidas_treino: list[ArrayLike], quantidade_epocas: int) -> None:
        """Chama self.treinar_uma_epoca ``quantidade_epocas`` vezes"""
        np.random.seed(self.seed)
        for epoca in range(quantidade_epocas):
            self.treinar_uma_epoca(taxa_aprendizado, tamanho_lotes, entradas_treino, saidas_treino)

In [4]:
rede = Rede_Neural([2,3,2], 42)

In [5]:
rede.previsao([1,2],-1)

array([-0.28570615, -2.26300906])

In [64]:
df = pd.read_excel("2018 Cruz Neto Polar wide.xlsx")
df = df.drop(columns=df.columns[0])
df = df.melt(id_vars=["Sample", "Pot (kW)", "Speed (mm/s)", "Thickness (mm)"],
             var_name="Theta (°)",
             value_name="Radius (mm)")

grupos = df.groupby("Sample")
grupos.get_group("2T-01")

,Sample,Pot (kW),Speed (mm/s),Thickness (mm),Theta (°),Radius (mm)
0,2T-01,4.053,6.67,6.3,0,4.211640
24,2T-01,4.053,6.67,6.3,1,4.150367
48,2T-01,4.053,6.67,6.3,2,4.029631
72,2T-01,4.053,6.67,6.3,3,3.791549
96,2T-01,4.053,6.67,6.3,4,3.721391
...,...,...,...,...,...,...
8544,2T-01,4.053,6.67,6.3,356,3.130822
8568,2T-01,4.053,6.67,6.3,357,3.319888
8592,2T-01,4.053,6.67,6.3,358,3.567133
8616,2T-01,4.053,6.67,6.3,359,4.083982


In [59]:
df_normalizado = df.copy()
df_normalizado[["Pot (kW)", "Speed (mm/s)", "Thickness (mm)", "Theta (°)", "Radius (mm)"]] = StandardScaler().fit_transform(df_normalizado[["Pot (kW)", "Speed (mm/s)", "Thickness (mm)", "Theta (°)", "Radius (mm)"]])
print(df_normalizado.drop(columns="Sample").min(), "\n")
print(df_normalizado.drop(columns="Sample").max())

Pot (kW)         -1.921290
Speed (mm/s)     -1.300740
Thickness (mm)   -1.364248
Theta (°)        -1.727260
Radius (mm)      -1.614410
dtype: float64 

Pot (kW)          1.364187
Speed (mm/s)      1.421644
Thickness (mm)    1.376163
Theta (°)         1.727260
Radius (mm)       3.977823
dtype: float64
